In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Mai 30

@author: yaning
"""

import numpy as np
import tensorflow as tf
# from tensorflow.keras import layers, models, datasets
import matplotlib.pyplot as plt
np.set_printoptions(threshold=np.inf)
import pickle
path = "/home/yaning/Documents/"

2025-12-03 17:17:40.223025: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-03 17:17:40.299248: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-03 17:17:40.364130: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764778660.425640  103058 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764778660.445155  103058 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764778660.576950  103058 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [7]:
with open(path + "Spiking_add_files/gabor_data_mnst.pkl", "rb") as f:
    normalised_gabor_data = pickle.load(f)

In [32]:
with open(path + "Spiking_add_files/fire_data_mnst_all_hunni.pkl", "rb") as f:
    normalised_gabor_data = pickle.load(f)

In [33]:
normalised_gabor_data.shape

(100, 10, 10, 8, 4000)

Creat 23 times only one picture

In [14]:
# form an shape (5000, 3) array with 5000 time points and 3 input neurons
# every fire is 45-50 cycles, 
pointCount = 6000
start_tstep = 500
one_cycle = np.concatenate((np.ones(45, dtype=bool), np.zeros(189, dtype=bool)))
# one_cycle = np.ones(234)
firing_duration = 45+189
# 23*(45+189) = 5382
total_time = 5382


non_fire_array = np.zeros(pointCount, dtype=bool)

fire_seq = np.tile(one_cycle, 23)
fire_array = non_fire_array.copy()
fire_array[start_tstep:start_tstep+len(fire_seq)] = fire_seq

In [15]:
normalised_gabor_data.shape

(30, 10, 10, 8)

In [16]:
fire_data = np.zeros((30,10,10,8, 4000), dtype=bool)
for i in range(normalised_gabor_data.shape[0]):
    for x in range(normalised_gabor_data.shape[1]):
        for y in range(normalised_gabor_data.shape[2]):
            for z in range(normalised_gabor_data.shape[3]):
                if normalised_gabor_data[i,x,y,z] == 0:
                    fire_data[i,x,y,z] = non_fire_array[:4000]
                else:
                    fire_data[i,x,y,z] = fire_array[:4000]

In [17]:
fire_data.shape

(30, 10, 10, 8, 4000)

Get the actual labels

In [16]:
with open("datasets/mnst_30_hunni.pkl", "rb") as f:
    imgs = pickle.load(f)

In [18]:
imgs["tag"][10]

np.uint8(1)

In [19]:
imgs["tag"][25]

np.uint8(8)

In [26]:
zero_labels = []
for i in range(imgs["tag"].shape[0]):
    if imgs["tag"][i] == 0:
        print(i)
        zero_labels.append(i)

4
7
21
26
33
38
39
45
51
58
65
78
84
88
89
91


In [25]:
nine_labels

[3, 13, 15, 18, 24, 27, 50, 57, 80, 86]

In [29]:
zero_labels

[4, 7, 21, 26, 33, 38, 39, 45, 51, 58]

In [35]:
nine_zero_labels = nine_labels + zero_labels

In [37]:
result = normalised_gabor_data[nine_zero_labels]

In [38]:
result.shape

(20, 10, 10, 8, 4000)

Create 23 times different pictures

In [7]:
normalised_gabor_data.shape

(30, 10, 10, 8)

In [10]:
nine_cluster = normalised_gabor_data[nine_labels]

In [23]:
nine_cluster.shape

(10, 10, 10, 8)

In [24]:
# pointCount = 6000
# start_tstep = 500
one_cycle = np.concatenate((np.ones(45, dtype=bool), np.zeros(189, dtype=bool)))

In [25]:
def from_gabor_to_fire(gabor):
    one_round = np.zeros((10,10,8,234), dtype=bool)
    for i in range(gabor.shape[0]):
        for j in range(gabor.shape[1]):
            for k in range(gabor.shape[2]):
                if gabor[i,j,k] == 1:
                    one_round[i,j,k] = one_cycle
    return one_round

In [26]:
fire_data = np.zeros((10,10,8,0), dtype=bool)
for i in nine_cluster:
    temp = from_gabor_to_fire(i)
    fire_data = np.concatenate((fire_data, temp), axis=-1)

In [29]:
# repeat all_three five times
repeat_fire_data = np.concatenate([fire_data] * 4, axis=-1)

In [30]:
repeat_fire_data.shape

(10, 10, 8, 9360)

In [31]:
# add a 500 start peace time
peace = np.zeros((10,10,8,500), dtype=bool)
fire_data = np.concatenate((peace, repeat_fire_data), axis=-1)

In [32]:
fire_data.shape

(10, 10, 8, 9860)

Save the fire data

In [24]:
chosen_fire_data = fire_data[22]

In [25]:
chosen_fire_data = np.concatenate([chosen_fire_data] * 4, axis=-1)

In [39]:
chosen_fire_data.shape

(10, 10, 8, 4000)

In [35]:
p = 0.01
mask = np.random.rand(*chosen_fire_data.shape) < p
b_chosen_fire_data = chosen_fire_data | mask

In [7]:
with open(path + "fire_data_mnst_all_second.pkl", "wb") as f:
    pickle.dump(fire_data, f)

In [25]:
with open(path + "fire_data_mnst_nine_first_30_twentisecond.pkl", "wb") as f:
    pickle.dump(chosen_fire_data, f)

In [39]:
with open(path + "Spiking_add_files/fire_data_nines_zeros.pkl", "wb") as f:
    pickle.dump(result, f)

In [21]:
fire_data.shape

(10, 10, 8, 5180)